In [10]:
import pandas as pd
from data_reading.phyphox import get_experiments, read_experiment
from preprocessing._interpolation import align_data

experiments = get_experiments()
offsets = {}
with open("../../data/phyphox/short recordings/Juan/offset.txt") as f:
    for line in f:
       (key, val) = line.split(": ")
       offsets[key] = val

sample_rate = 50
data_frames = read_experiment(experiments[1], offsets=offsets)
data_frames = {key : align_data(data_frame, listening_rate=1000/sample_rate, reference_sensor=None) for key, data_frame in data_frames.items()}

In [12]:
# TODO: load annotations
y = pd.read_csv("../../data/phyphox/short recordings/Juan/annotations.tsv", delimiter="\t")
y = y.iloc[:, [3,5,8]]
y.columns = ["start", "end", "label"]
# iterate over the annotations and split the timeseries in chunks
chunks = {}
for key, df in data_frames.items():
    chunks[key] = [df.iloc[int(annotation["start"]*sample_rate):int(annotation["end"]*sample_rate)] for i, annotation in y.iterrows()]

                 linear_acceleration x  linear_acceleration y  \
00:00:52.760383               0.146769               0.061971   
00:00:52.780383               0.048665              -0.039880   
00:00:52.800383               0.223883              -0.192375   
00:00:52.820383               0.376531              -0.161761   
00:00:52.840383               0.169982              -0.000448   
...                                ...                    ...   
00:00:57.520383               0.745064               0.314071   
00:00:57.540383              -0.141403               0.033819   
00:00:57.560383              -0.564179               0.194906   
00:00:57.580383              -0.736732               0.156795   
00:00:57.600383              -0.856961               0.008764   

                 linear_acceleration z  gyroscope x  gyroscope y  gyroscope z  \
00:00:52.760383              -0.034756    -0.002906     0.054267     0.126520   
00:00:52.780383               0.135449    -0.003591     0

In [21]:
# append the activity label (as int) and the action id to the dataframe
# we need to do this to be able to extract time series features later

labels = y.loc[:, "label"].unique()
label_ids = { l: i for l,i in zip(labels, range(0,len(labels))) }

y = y.replace(label_ids)
for i, cl in zip(range(len(y)), y.iterrows()):
    label_id = int(cl[1]["label"])
    action_id = cl[0]
    for hand, chunk_list in chunks.items():
        c = chunk_list[i]
        c["activity"] = label_id
        c["action_id"] = action_id

/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
chunks_concat_right = pd.concat(chunks["right"]).reset_index(drop=True)
chunks_concat_left = pd.concat(chunks["left"]).reset_index(drop=True)


In [23]:
%load_ext autoreload
%autoreload 2

labels = y.loc[:, "label"].squeeze()
from features._timeseries_feature_extraction import extract_timeseries_features
features = extract_timeseries_features(chunks_concat_right, labels)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Feature Extraction: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]


In [24]:
features.dropna(inplace=True, axis=1)
features

variable,acceleration x__abs_energy,acceleration x__absolute_sum_of_changes,"acceleration x__agg_autocorrelation__f_agg_""mean""__maxlag_40","acceleration x__agg_autocorrelation__f_agg_""median""__maxlag_40","acceleration x__agg_autocorrelation__f_agg_""var""__maxlag_40","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,linear_acceleration z__symmetry_looking__r_0.9500000000000001,linear_acceleration z__time_reversal_asymmetry_statistic__lag_1,linear_acceleration z__time_reversal_asymmetry_statistic__lag_2,linear_acceleration z__time_reversal_asymmetry_statistic__lag_3,linear_acceleration z__value_count__value_-1,linear_acceleration z__value_count__value_0,linear_acceleration z__value_count__value_1,linear_acceleration z__variance,linear_acceleration z__variance_larger_than_standard_deviation,linear_acceleration z__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
0,6956.588974,131.413302,0.462771,0.433799,0.032429,8.930229,7.073781,5.202037,2.060620,10.434962,...,1.0,0.429494,-0.164980,0.597156,0.0,0.0,0.0,0.820118,0.0,-23.569820
1,8104.145071,133.492186,0.201058,0.188168,0.040767,8.197083,6.664591,5.545273,1.760806,10.093124,...,1.0,0.001389,0.280241,-0.246189,0.0,0.0,0.0,0.764085,0.0,-41.794496
2,4605.980768,90.490004,0.166964,0.151822,0.061212,5.171849,4.624486,4.201828,-0.403371,7.352635,...,1.0,0.762612,0.606624,-0.268059,0.0,0.0,0.0,2.633105,1.0,-12.511400
3,6726.574496,124.109812,0.530803,0.486180,0.027969,8.726986,6.704075,5.091302,1.793982,11.576229,...,1.0,0.031047,0.050746,0.043174,0.0,0.0,0.0,0.586616,0.0,-3.447960
4,37530.307701,112.825301,0.085717,0.205488,0.370038,10.456120,9.765932,9.264976,0.208281,11.114417,...,1.0,0.024199,-0.003485,-0.113954,0.0,0.0,0.0,0.751023,0.0,-2.155912
5,12310.916475,33.580562,0.082059,0.161694,0.398732,11.100144,9.971857,8.810321,0.538450,12.158878,...,1.0,-0.062139,-0.071586,-0.106532,0.0,0.0,0.0,2.832656,1.0,-5.377884
6,12993.208635,58.873554,0.628405,0.586513,0.020766,10.571669,8.934102,7.543843,1.221601,12.302055,...,1.0,-0.003978,-0.026251,-0.073435,0.0,0.0,0.0,2.056634,1.0,-3.122216
7,4958.295813,88.954265,0.213288,0.042454,0.123990,3.017792,2.193023,1.327690,0.439155,4.934755,...,1.0,-0.049494,-0.135443,-0.251444,0.0,0.0,0.0,2.483115,1.0,-3.116841
8,3389.079635,49.194103,0.079773,-0.037752,0.123117,3.591803,3.115080,2.713205,0.153451,4.406817,...,1.0,-0.162477,-0.231394,-0.295534,0.0,0.0,0.0,3.177103,1.0,-6.893696


In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [('Logistic Regression', LogisticRegression(solver='liblinear', multi_class='ovr')), ('LDA', LinearDiscriminantAnalysis()), ('LinearSVC', LinearSVC()), ('CART', DecisionTreeClassifier()), ('NB', GaussianNB())]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
for name, model in models:
    print(name)
    scores = cross_val_score(model, X_scaled, labels, cv=2)
    print(scores)

Logistic Regression
[0.4375     0.53333333]
LDA
[0.625 0.2  ]
LinearSVC
[0.4375     0.46666667]
CART
[0.25       0.06666667]
NB
[0.25       0.26666667]


/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n